In [0]:
from pyspark.sql import functions as F

BASE = "/Volumes/raw/sistema_pontos"

TABLES = [
    {
        "name": "clientes",
        "pk": "idCliente",
        "date_field": "DtAtualizacao"   # tem update
    },
    {
        "name": "produtos",
        "pk": "IdProduto",
        "date_field": None              #  sem update
    },
    {
        "name": "transacoes",
        "pk": "IdTransacao",
        "date_field": None              #  só DtCriacao
    },
    {
        "name": "transacao_produto",
        "pk": "idTransacaoProduto",
        "date_field": None              #  sem update
    }
]


In [0]:
def create_cdc(table_name: str, pk: str, date_field: str | None):

    path_baseline = f"{BASE}/baseline/{table_name}"
    path_actual   = f"{BASE}/full_load/{table_name}"
    path_cdc      = f"{BASE}/cdc/{table_name}"

    df_last = spark.read.parquet(path_baseline)
    df_act  = spark.read.parquet(path_actual)

    # ➕ INSERT (pk só no atual)
    inserts = (
        df_act.alias("a")
        .join(df_last.select(pk).alias("l"), on=pk, how="left_anti")
        .select("a.*")
        .withColumn("op", F.lit("I"))
    )

    # ➖ DELETE (pk só no baseline)
    deletes = (
        df_last.alias("l")
        .join(df_act.select(pk).alias("a"), on=pk, how="left_anti")
        .select("l.*")
        .withColumn("op", F.lit("D"))
    )

    frames = [inserts, deletes]

    # 🔄 UPDATE (somente se existir date_field)
    if date_field:
        df_last = df_last.withColumn(
            date_field, F.to_timestamp(F.col(date_field))
        )
        df_act = df_act.withColumn(
            date_field, F.to_timestamp(F.col(date_field))
        )

        updates = (
            df_last.alias("l")
            .join(df_act.alias("a"), on=pk, how="inner")
            .where(F.col(f"a.{date_field}") > F.col(f"l.{date_field}"))
            .select("a.*")
            .withColumn("op", F.lit("U"))
        )

        frames.append(updates)

    cdc = (
        frames[0]
        .unionByName(frames[1])
        .unionByName(frames[2]) if len(frames) == 3 else
        frames[0].unionByName(frames[1])
    ).withColumn("_cdc_ts", F.current_timestamp())

    if cdc.limit(1).count() == 0:
        print(f"ℹ️ {table_name}: nenhuma alteração")
        return

    (
        cdc.coalesce(1)
        .write
        .mode("append")
        .format("parquet")
        .save(path_cdc)
    )

    print(f"✅ CDC gerado: {table_name}")


In [0]:
for t in TABLES:
    create_cdc(
        table_name=t["name"],
        pk=t["pk"],
        date_field=t["date_field"]
    )

# 🔁 baseline = snapshot atual
for t in TABLES:
    name = t["name"]
    (
        spark.read.parquet(f"{BASE}/full_load/{name}")
        .coalesce(1)
        .write
        .mode("overwrite")
        .parquet(f"{BASE}/baseline/{name}")
    )

print("🔁 baseline atualizado com sucesso")
